In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

In [17]:
def get_prospects_for_year(year):
    prospects = []
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    try:
        driver.get(f"https://www.tankathon.com/past_drafts/{year}")
        time.sleep(2)  
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        rows = soup.find_all('div', class_="mock-row")

        for row in rows:
            try:
                name = row.find('div', class_="mock-row-name")
                # raw position_school is in format "position | school"
                position_school = row.find('div', class_="mock-row-school-position")
                team = position_school.text.split("|")[1].strip() 

                year_age = row.find('div', class_="section year-age desktop")
                classification = year_age.find_all('div')[0].text.strip()
                if classification in ['Freshman', 'Sophomore', 'Junior', 'Senior']:
                    classification = "College"

                prospects.append({
                    'name': name.text.strip(),
                    'team': team,
                    'classification': classification
                })
            except Exception as e:
                print(f"Error: {e}")
                continue

        prospects_df = pd.DataFrame(prospects)
        prospects_df.to_csv(f"../data/raw/prospect_pool_{year}.csv", index=False)                
        return prospects_df
        
    finally:
        driver.quit()

In [18]:
# 2024 testing
get_prospects_for_year(2024)

Error: 'NoneType' object has no attribute 'text'
Error: 'NoneType' object has no attribute 'text'


,name,team,classification
0,Zaccharie Risacher,JL Bourg,International
1,Alexandre Sarr,Perth,International
2,Reed Sheppard,Kentucky,College
3,Stephon Castle,UConn,College
4,Ron Holland,G League,G League
...,...,...,...
69,Isaac Jones,Washington State,College
70,Judah Mintz,Syracuse,College
71,Ousmane N'Diaye,Saski Baskonia B,International
72,Zyon Pullin,Florida,College
